In [182]:
import pandas as pd

In [183]:
def calc_delta_m(base_acc_nap: float, base_error_ntp: float, base_error_rtp: float, acc_nap: float, error_ntp: float, error_rtp: float):
    """Calculates delta-M metric
    """

    tasks = []
    if acc_nap is not None and not pd.isna(acc_nap):
        tasks.append(acc_nap)
    if error_ntp is not None and not pd.isna(error_ntp):
        tasks.append(error_ntp)
    if error_rtp is not None and not pd.isna(error_rtp):
        tasks.append(error_rtp)

    delta_m = (
        1 / len(tasks)
        * (
            ((-1) * (acc_nap - base_acc_nap) / base_acc_nap if acc_nap is not None and not pd.isna(acc_nap) else 0)
            + (1 * (error_ntp - base_error_ntp) / base_error_ntp if error_ntp is not None and not pd.isna(error_ntp) else 0)
            + (1 * (error_rtp - base_error_rtp) / base_error_rtp if error_rtp is not None and not pd.isna(error_rtp) else 0)
        )
    )
    return (delta_m * 100).round(2)

In [181]:
# (1/3 * (((-1) * (0.28-0.11) / 0.11) + (1 * (2.09-2.08) / 2.08) + (1 * (32.04 - 35.15) / 35.15)))*100

In [188]:
# df_original = pd.read_csv("P2P/P2P_best_results.csv")
# df_original = pd.read_csv("Production/Production_best_results.csv")
# df_original = pd.read_csv("HelpDesk/HelpDesk_best_results.csv")
# df_original = pd.read_csv("Sepsis/Sepsis_best_results.csv")
# df_original = pd.read_csv("BPIC20_DomesticDeclarations/BPIC20_DomesticDeclarations_best_results.csv")
# df_original = pd.read_csv("Sepsis_small/Sepsis_small_best_results.csv")
df_original = pd.read_csv("BPI_Challenge_2013_incidents/BPI_Challenge_2013_incidents_best_results.csv")
DATASET = "BPIC13I"
df_original

,Model,MTL,Tasks,Learning Rate,MTL HPO,NEXT_ACTIVITY_mean,NEXT_ACTIVITY_std,NEXT_TIME_mean,NEXT_TIME_std,REMAINING_TIME_mean,REMAINING_TIME_std,Best Epoch_mean
0,LSTM,UW_SO,"('next_activity', 'next_time', 'remaining_time')",0.0001,3.0,0.644067,0.005774,1.319167,0.065353,9.280500,0.097945,196.666667
1,LSTM,RLW,"('next_activity', 'next_time', 'remaining_time')",0.0001,NaN,0.634267,0.008737,1.354467,0.048420,10.090967,0.484386,149.000000
2,LSTM,CAGrad,"('next_activity', 'next_time', 'remaining_time')",0.0001,0.1,0.649767,0.009450,1.324833,0.067476,9.316167,0.228022,197.000000
3,LSTM,EW,"('next_activity', 'next_time', 'remaining_time')",0.0001,NaN,0.638700,0.007408,1.337433,0.059213,9.319233,0.313145,185.000000
4,LSTM,GradDrop,"('next_activity', 'next_time', 'remaining_time')",0.0001,NaN,0.633800,0.007300,1.347300,0.058868,9.465733,0.368795,180.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
118,Transformer,UW,"('next_activity', 'next_time', 'remaining_time')",0.0001,NaN,0.696367,0.013267,1.300600,0.085389,6.735333,0.300282,45.333333
119,Transformer,IMTL,"('next_activity', 'next_time', 'remaining_time')",0.0001,NaN,0.708167,0.010773,1.332700,0.106129,7.317600,0.168314,39.666667
120,Transformer,EW,"('next_activity',)",0.0010,NaN,0.594100,0.049796,NaN,NaN,NaN,NaN,31.666667
121,Transformer,EW,"('remaining_time',)",0.0001,NaN,NaN,NaN,NaN,NaN,6.768567,0.597980,29.333333


In [170]:
# # remove row where MTL == IMTL
# df_original = df_original[df_original['MTL'] != 'IMTL'].reset_index()
# df_original

In [189]:
unique_task_combinations = list(df_original['Tasks'].unique())
# remove stl tasks
unique_task_combinations.remove("('next_activity',)")
unique_task_combinations.remove("('remaining_time',)")
unique_task_combinations.remove("('next_time',)")

nap_task = "('next_activity',)"
ntp_task = "('next_time',)"
rtp_task = "('remaining_time',)"
ts = [nap_task, ntp_task, rtp_task]
ts_names_mean = ['NEXT_ACTIVITY_mean', 'NEXT_TIME_mean', 'REMAINING_TIME_mean']
ts_names_std = ['NEXT_ACTIVITY_std', 'NEXT_TIME_std', 'REMAINING_TIME_std']

unique_architectures = df_original['Model'].unique()

for architecture in unique_architectures:
    print(f"architecture: {architecture}")
    # get STL results for this architecture
    stl_row = {'MTO': 'STL'}
    i = 0
    base_acc_nap = None
    base_error_ntp = None
    base_error_rtp = None

    for t in ts:
        filtered_df = df_original[(df_original['Tasks'] == t) & (df_original['Model'] == architecture)]
        mean = filtered_df[ts_names_mean[i]].values[0]
        std = filtered_df[ts_names_std[i]].values[0]
        if 'activity' in t:
            stl_row['NAP'] = f"{mean:.2f} ({std:.2f})"
            base_acc_nap = mean
        elif 'next_time' in t:
            stl_row['NTP'] = f"{mean:.2f} ({std:.2f})"
            base_error_ntp = mean
        else:
            stl_row['RTP'] = f"{mean:.2f} ({std:.2f})"
            base_error_rtp = mean
        i += 1
    stl_row['Best Epoch'] = "/"
    stl_row['$\Delta_m$'] = "/"

    # Apply row-wise and assign to the column
    df_original['$\Delta_m$'] = df_original.apply(
        lambda row: calc_delta_m(
            base_acc_nap=base_acc_nap,
            base_error_ntp=base_error_ntp,
            base_error_rtp=base_error_rtp,
            acc_nap=row['NEXT_ACTIVITY_mean'],
            error_ntp=row['NEXT_TIME_mean'],
            error_rtp=row['REMAINING_TIME_mean']
        ),
        axis=1
    )

    for task_combination in unique_task_combinations:
        df = df_original.copy()
        stl_row_copy = stl_row.copy()
        # rename some method names in the MTL column such as 'UW_SO' to 'UW-SO'
        df['MTL'] = df['MTL'].replace('UW_SO', 'UW-SO')
        df['MTL'] = df['MTL'].replace('UW_O', 'UW-O')
        df['MTL'] = df['MTL'].replace('Nash_MTL', 'NashMTL')

        print(f"task_combination: {task_combination}")
        task_combi = ""
        df = df[(df['Tasks'] == task_combination) & (df['Model'] == architecture)]
        # add new columns
        if not df['NEXT_ACTIVITY_mean'].isna().any():
            df['NAP'] = df.apply(
                lambda row: f"{row['NEXT_ACTIVITY_mean']:.2f} ({row['NEXT_ACTIVITY_std']:.2f})",
                axis=1
            )
            task_combi += "NAP+"
        else:
            # remove NAP in stl_row dict
            stl_row_copy.pop('NAP')
        if not df['NEXT_TIME_mean'].isna().any():
            df['NTP'] = df.apply(
                lambda row: f"{row['NEXT_TIME_mean']:.2f} ({row['NEXT_TIME_std']:.2f})",
                axis=1
            )
            task_combi += "NTP+"
        else:
            # remove NTP in stl_row dict
            stl_row_copy.pop('NTP')
        if not df['REMAINING_TIME_mean'].isna().any():
            df['RTP'] = df.apply(
                lambda row: f"{row['REMAINING_TIME_mean']:.2f} ({row['REMAINING_TIME_std']:.2f})",
                axis=1
            )
            task_combi += "RTP"
        else:
            # remove RTP in stl_row dict
            stl_row_copy.pop('RTP')
        if task_combi[-1] == "+":
            task_combi = task_combi[:-1]
        # rename best epoch column
        df = df.rename(columns={'Best Epoch_mean': 'Best Epoch'})
        df['Best Epoch'] = df.apply(
            lambda row: f"{row['Best Epoch']:.0f}",
            axis=1
        )

        df = df.rename(columns={'$\Delta_m$': 'Delta_m'})
        df['Delta_m'] = df.apply(
            lambda row: f"{row['Delta_m']:.2f}",
            axis=1
        )
        df = df.rename(columns={'Delta_m': '$\Delta_m$'})

        # rename MTL column
        df = df.rename(columns={'MTL': 'MTO'})
        # remove columns
        df = df.drop(columns=['Tasks', 'Model', 'Learning Rate', 'NEXT_ACTIVITY_mean', 'NEXT_TIME_mean', 'REMAINING_TIME_mean', 'NEXT_ACTIVITY_std', 'NEXT_TIME_std', 'REMAINING_TIME_std', 'MTL HPO'])

        # add STL results in one line at the top
        df = pd.concat([pd.DataFrame([stl_row_copy]), df])

        # reorder the rows depending on the MTO method such that the following order is respected:
        mto_order = ['STL', 'EW', 'Scalarization', 'DWA', 'RLW', 'UW', 'UW-O', 'UW-SO', 'GLS', 'IMTL', 'GradNorm', 'NashMTL', 'GradDrop', 'PCGrad', 'CAGrad']
        if 'Scalarization' not in df['MTO'].values:
            mto_order.remove('Scalarization')
        # reorder based on mto_order
        df = df.sort_values(by='MTO', key=lambda x: x.map(lambda y: mto_order.index(y)))



        latex_tabular = df.to_latex(
            index=False,
            bold_rows=False,
            column_format="lrrrrr",

        )

        # Wrap in table* and remove indentation
        latex_full = f"\\begin{{table*}}[htbp] \\caption{{Results on the {DATASET} dataset for {architecture} with task combination {task_combi}}} \\label{{tab:{DATASET}_results_{architecture}_{task_combi}}} \\centering {latex_tabular}\\end{{table*}}"
        # latex_full = f"\\begin{{table*}}[htbp] \\caption{{Results on the {DATASET} dataset for the simplified {architecture} model with task combination {task_combi}}} \\label{{tab:{DATASET}_results_{architecture}_{task_combi}}} \\centering {latex_tabular}\\end{{table*}}"

        print(latex_full)
        print("#################")


architecture: LSTM
task_combination: ('next_activity', 'next_time', 'remaining_time')
\begin{table*}[htbp] \caption{Results on the BPIC13I dataset for LSTM with task combination NAP+NTP+RTP} \label{tab:BPIC13I_results_LSTM_NAP+NTP+RTP} \centering \begin{tabular}{lrrrrr}
\toprule
MTO & NAP & NTP & RTP & Best Epoch & $\Delta_m$ \\
\midrule
STL & 0.52 (0.00) & 1.40 (0.06) & 9.65 (0.30) & / & / \\
EW & 0.64 (0.01) & 1.34 (0.06) & 9.32 (0.31) & 185 & -10.69 \\
DWA & 0.64 (0.00) & 1.33 (0.07) & 9.22 (0.15) & 198 & -11.23 \\
RLW & 0.63 (0.01) & 1.35 (0.05) & 10.09 (0.48) & 149 & -7.33 \\
UW & 0.66 (0.00) & 1.31 (0.07) & 9.49 (0.11) & 197 & -12.16 \\
UW-O & 0.66 (0.01) & 1.32 (0.06) & 10.73 (0.68) & 153 & -7.52 \\
UW-SO & 0.64 (0.01) & 1.32 (0.07) & 9.28 (0.10) & 197 & -11.60 \\
GLS & 0.66 (0.00) & 1.30 (0.07) & 10.23 (0.55) & 166 & -10.02 \\
IMTL & 0.56 (0.06) & 1.54 (0.05) & 12.17 (0.26) & 35 & 8.99 \\
GradNorm & 0.66 (0.00) & 1.32 (0.07) & 9.41 (0.17) & 198 & -12.25 \\
NashMTL & 0.64 (0.01)